# Prototype of BIS2 project 

In [8]:
import geopandas
import pandas
import geojson
import requests
import json
from shapely.geometry import Point
import geopandas as gpd 
from shapely.geometry import Polygon

## Input
### Here we take the input from group 4. Obviously in this semplified version the input is just a point with the coordinates of Milan.

In [9]:
lat_point_list = [50.854457, 52.518172, 50.072651, 48.853033, 50.854457]
lon_point_list = [4.377184, 13.407759, 14.435935, 2.349553, 4.377184]

polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[polygon_geom])

In [10]:
lat = 45.464664
lon = 9.188540
id = 10

In [11]:
point = Point(lat, lon)

## API request
### The API request is executed with open-meteo.com

In [12]:
BASE_URL = "https://api.open-meteo.com/v1/forecast?latitude=" + str(lat) + "&longitude=" + str(lon) + "&hourly=temperature_2m,precipitation_probability,cloudcover,visibility,windspeed_10m,windspeed_180m,is_day"

response = requests.get(BASE_URL)

response = response.json()

print(json.dumps(response, indent=4))

{
    "latitude": 45.46,
    "longitude": 9.18,
    "generationtime_ms": 0.635981559753418,
    "utc_offset_seconds": 0,
    "timezone": "GMT",
    "timezone_abbreviation": "GMT",
    "elevation": 142.0,
    "hourly_units": {
        "time": "iso8601",
        "temperature_2m": "\u00b0C",
        "precipitation_probability": "%",
        "cloudcover": "%",
        "visibility": "m",
        "windspeed_10m": "km/h",
        "windspeed_180m": "km/h",
        "is_day": ""
    },
    "hourly": {
        "time": [
            "2023-05-10T00:00",
            "2023-05-10T01:00",
            "2023-05-10T02:00",
            "2023-05-10T03:00",
            "2023-05-10T04:00",
            "2023-05-10T05:00",
            "2023-05-10T06:00",
            "2023-05-10T07:00",
            "2023-05-10T08:00",
            "2023-05-10T09:00",
            "2023-05-10T10:00",
            "2023-05-10T11:00",
            "2023-05-10T12:00",
            "2023-05-10T13:00",
            "2023-05-10T14:00",
     

## Parsing of metereological data
### In this section the GeoPandas geoDataframe is built

In [13]:
gdf = geopandas.GeoDataFrame()

gdf['time'] = pandas.to_datetime(response['hourly']['time']).strftime('%Y-%m-%dT%H:%M:%S')
gdf['AOI_ID'] = id
gdf['EventID'] = id

gdf['temperature'] = response['hourly']['temperature_2m']
gdf['precipitationProbability'] = response['hourly']['precipitation_probability']
gdf['cloudcover'] =  response['hourly']['cloudcover']
gdf['day/night'] = response['hourly']['is_day']
gdf['geometry'] = polygon_geom
gdf['centroid'] = polygon_geom.centroid

gdf

,time,AOI_ID,EventID,temperature,precipitationProbability,cloudcover,day/night,centroid
0,2023-05-10T00:00:00,10,10,14.3,100,100,0,POINT (8.981408476975249 50.56546094509109)
1,2023-05-10T01:00:00,10,10,14.2,100,100,0,POINT (8.981408476975249 50.56546094509109)
2,2023-05-10T02:00:00,10,10,14.1,100,100,0,POINT (8.981408476975249 50.56546094509109)
3,2023-05-10T03:00:00,10,10,13.8,100,100,0,POINT (8.981408476975249 50.56546094509109)
4,2023-05-10T04:00:00,10,10,13.1,100,100,1,POINT (8.981408476975249 50.56546094509109)
...,...,...,...,...,...,...,...,...
163,2023-05-16T19:00:00,10,10,19.7,67,99,0,POINT (8.981408476975249 50.56546094509109)
164,2023-05-16T20:00:00,10,10,18.7,75,99,0,POINT (8.981408476975249 50.56546094509109)
165,2023-05-16T21:00:00,10,10,17.4,84,98,0,POINT (8.981408476975249 50.56546094509109)
166,2023-05-16T22:00:00,10,10,16.8,84,92,0,POINT (8.981408476975249 50.56546094509109)


## Export
### At this point the ouput is created

In [14]:

# print(json_gdf)
# print(json.dumps(json_gdf))
# gdf['geometry'] = polygon_geom.__geo_interface__
gdf['centroid'] = json.dumps(point.__geo_interface__)

json_gdf = gdf.to_json()

print(gdf['centroid'])

x = json.loads(json_gdf)
print(json.dumps(x, indent=4))

# with open('my_file.json', 'w') as f:
#     f.write(json_gdf)


    


AttributeError: No geometry data set (expected in column 'geometry').

In [ ]:
y = geojson.loads(json_gdf)
print(json.dumps(y, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "id": "0",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            4.377184,
                            50.854457
                        ],
                        [
                            13.407759,
                            52.518172
                        ],
                        [
                            14.435935,
                            50.072651
                        ],
                        [
                            2.349553,
                            48.853033
                        ],
                        [
                            4.377184,
                            50.854457
                        ]
                    ]
                ]
            },
            "properties": {
                "time": "2023-05-03T00